In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
#chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Downloads\Compressed\chromedriver_win32\chromedriver.exe',options=chrome_options)

In [3]:
url='https://forumevents.co.uk/all-events/'

driver.get(url)
time.sleep(5)

soup=bs(driver.page_source,'lxml')
cars=soup.find_all('article',{'class':re.compile('grve-isotope-item grve-portfolio-item.+')})

len(cars)

cards=[]
for j in cars:
    link=j.find('a')['href']
    info=j.find('h6',{'class':'grve-title grve-dark'})
    
    cards.append([link,info])



In [7]:
forev=[]
for i in cards:
    link=i[0]
    event_info=i[1]

    driver.get(link)
    soupn=bs(driver.page_source,'lxml')
    title=' '.join(soupn.find('title').text.split()).split('|')[0].strip()
    try:
        dat=' '.join(soupn.find('div',{'class':re.compile('grve-description grve-ligh.+')}).get_text(separator='{').split()).split('{')[-2].strip()
    except:
        try:
            dat=' '.join(soupn.find('rs-layer',{'class':re.compile('Fashion-TextBlock rs-laye.+')}).get_text(separator='{').split()).split('{')[-2].strip()
        except:
            dat=''
            
    try:
        loc=' '.join(soupn.find('div',{'class':re.compile('grve-description grve-ligh.+')}).get_text(separator='{').split()).split('{')[-1].strip().title()
    except:
        try:
            loc=' '.join(soupn.find('rs-layer',{'class':re.compile('Fashion-TextBlock rs-laye.+')}).get_text(separator='{').split()).split('{')[-1].strip().title()
        except:
            loc=''
            
    if dat!='':
        if not re.search('\d{4}',dat):
            if re.search('\d{4}',loc):
                dat=loc
            else:
                dat=''
    if dat!='':
        dat=re.search('\d+\w+\s+\&?\s*\d*\w*\s*[A-Za-z]+\s*\d{4}',dat).group().title()
    
    if re.search('\d{4}',loc):
        loc=loc.replace(re.search('\d+\w+\s+\&?\s*\d*\w*\s*[A-Za-z]+\s*\d{4}',loc).group(),'').replace('–','').replace('-','').strip().title()
    
    dat=dat.replace('–','-').replace('|','').replace('to','-').replace('TO','-').replace('c-ber','ctober').replace('C-BER','CTOBER').strip()
    if '(' in dat:
        dat=dat.split('(')[0].strip()
    if dat=='':
        start_date=end_date=''
    else:
        mad=dat
    if '-' in mad:
        st=mad.split('-')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('-')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif 'and'in mad:
        st=mad.split('and')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('and')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif '&' in mad:
        st=mad.split('&')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('&')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    else:
        st=mad.strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
   
    if any(c.isalpha() for c in st)==False:
    #      print('leg')
        pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en
    #                        02        April      2022
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
     #   print('awujale')
        start=st
        end=en

    #*                        April       02      2022
    elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
    #       print('ala')
        pa=re.search('(\d{4})',st).group()
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()
        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba     

    #                      02     2022
    elif re.search('\d{1,2}\s+\d{4}',en):
    #      print('kum')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',st).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    #                      02         April
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
    #        print('is')
        pa=re.search('(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en

    #*                      April           02
    elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
    #      print('bad')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    else:
    #     print('shik')
        start=end=''
    if start=='':
        start_date=end_date=''
    else:
        pick=[start, end]
        try:
            spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
        except:
            spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]


#     #**************
    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            try:
                return(gu(location))
            except:
                sha=location.split(',')
                try:
                    gu(sha[-3])
                except:
                    try:
                        gu(sha[-2])
                    except:
                        try:
                            gu(sha[-1])
                        except Exception as e:
                            print(location, "; url didn't go through")
                            print(e)
                            return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                    except:
                        google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                        time.sleep(randint(0,3))
                        driver.get(google_url_for_location)
                        time.sleep(4)
                        google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")

    time_=''
    org_name='The Forum Experience'
    org_web='https://forumevents.co.uk/'
    org_pro='The Forum Experience connects the corporate business world through multiple sectors, with the vision of generating lasting business relationships via a series of pre-arranged meetings.'
    logo=''    
    sponsor=''
    agendalist=''
    type_=''
    category=''
    Speakerlist='' 
    ticket_list=''
    event_web=link
    mail_=json.dumps(['enquiries@forumevents.co.uk'])

    if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc:
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        on_off='0'
        if ',' in loc:
            city=loc.split(',')[-1].strip()
        
        else:
            city=loc#.split()[-3]
        country=countr(city)
        venue=loc
        googlePlaceUrl=get_google_map_url(venue)
    
            
    
    forev.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])


Radisson Hotel & Conference Centre London Heathrow ;country didn't go through
Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="lu_map"]"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BC6463+2188387]
	Ordinal0 [0x00B5E461+1762401]
	Ordinal0 [0x00A73D78+802168]
	Ordinal0 [0x00AA1880+989312]
	Ordinal0 [0x00AA1B1B+989979]
	Ordinal0 [0x00ACE912+1173778]
	Ordinal0 [0x00ABC824+1099812]
	Ordinal0 [0x00ACCC22+1166370]
	Ordinal0 [0x00ABC5F6+1099254]
	Ordinal0 [0x00A96BE0+945120]
	Ordinal0 [0x00A97AD6+948950]
	GetHandleVerifier [0x00E671F2+2712546]
	GetHandleVerifier [0x00E5886D+2652765]
	GetHandleVerifier [0x00C5002A+520730]
	GetHandleVerifier [0x00C4EE06+516086]
	Ordinal0 [0x00B6468B+1787531]
	Ordinal0 [0x00B68E88+1805960]
	Ordinal0 [0x00B68F75+1806197]
	Ordinal0 [0x00B71DF1+1842673]
	BaseThreadInitThunk [0x7602FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77077A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77077A6E+

RADISSON HOTEL & CONFERENCE CENTRE LONDON HEATHROW ;country didn't go through
Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="lu_map"]"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BC6463+2188387]
	Ordinal0 [0x00B5E461+1762401]
	Ordinal0 [0x00A73D78+802168]
	Ordinal0 [0x00AA1880+989312]
	Ordinal0 [0x00AA1B1B+989979]
	Ordinal0 [0x00ACE912+1173778]
	Ordinal0 [0x00ABC824+1099812]
	Ordinal0 [0x00ACCC22+1166370]
	Ordinal0 [0x00ABC5F6+1099254]
	Ordinal0 [0x00A96BE0+945120]
	Ordinal0 [0x00A97AD6+948950]
	GetHandleVerifier [0x00E671F2+2712546]
	GetHandleVerifier [0x00E5886D+2652765]
	GetHandleVerifier [0x00C5002A+520730]
	GetHandleVerifier [0x00C4EE06+516086]
	Ordinal0 [0x00B6468B+1787531]
	Ordinal0 [0x00B68E88+1805960]
	Ordinal0 [0x00B68F75+1806197]
	Ordinal0 [0x00B71DF1+1842673]
	BaseThreadInitThunk [0x7602FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77077A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77077A6E+

WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BC6463+2188387]
	Ordinal0 [0x00B5E461+1762401]
	Ordinal0 [0x00A73D78+802168]
	Ordinal0 [0x00A67210+750096]
	Ordinal0 [0x00A6675A+747354]
	Ordinal0 [0x00A65D3F+744767]
	Ordinal0 [0x00A64C28+740392]
	Ordinal0 [0x00A65228+741928]
	Ordinal0 [0x00A6EF2F+782127]
	Ordinal0 [0x00A79FBB+827323]
	Ordinal0 [0x00A7D310+840464]
	Ordinal0 [0x00A654F6+742646]
	Ordinal0 [0x00A79BF3+826355]
	Ordinal0 [0x00ACCF6D+1167213]
	Ordinal0 [0x00ABC5F6+1099254]
	Ordinal0 [0x00A96BE0+945120]
	Ordinal0 [0x00A97AD6+948950]
	GetHandleVerifier [0x00E671F2+2712546]
	GetHandleVerifier [0x00E5886D+2652765]
	GetHandleVerifier [0x00C5002A+520730]
	GetHandleVerifier [0x00C4EE06+516086]
	Ordinal0 [0x00B6468B+1787531]
	Ordinal0 [0x00B68E88+1805960]
	Ordinal0 [0x00B68F75+1806197]
	Ordinal0 [0x00B71DF1+1842673]
	BaseThreadInitTh

In [8]:
len(forev)

22

In [9]:
forev

[['https://forumevents.co.uk/events/building-construction-summit/',
  'Building & Construction Summit',
  '2023-03-20',
  '2023-03-20',
  '',
  <h6 class="grve-title grve-dark"></h6>,
  '',
  'The Forum Experience connects the corporate business world through multiple sectors, with the vision of generating lasting business relationships via a series of pre-arranged meetings.',
  'The Forum Experience',
  'https://forumevents.co.uk/',
  '',
  '',
  '',
  '',
  '',
  'Hilton London Canary Wharf',
  '',
  'Hilton London Canary Wharf',
  'https://forumevents.co.uk/events/building-construction-summit/',
  'https://www.google.com/maps/place/Hilton+London+Canary+Wharf/@51.5009232,-0.0197283,15z/data=!4m2!3m1!1s0x0:0xab02770094382c7f?sa=X&ved=2ahUKEwipo-nj-eL4AhW1VfEDHX5NCuwQ_BJ6BAghEAU',
  '["enquiries@forumevents.co.uk"]',
  '',
  '0'],
 ['https://forumevents.co.uk/events/care-forum/',
  'The Care Forum',
  '2023-06-05',
  '2023-06-06',
  '',
  <h6 class="grve-title grve-dark"></h6>,
  '',
 

In [10]:
forev_dict=forev


forev_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=forev_dict)
forev_df.to_csv("forev.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [ ]:
# s=28
# for k in cards[27:]:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[0])
#     s+=1
# print(s)

In [ ]:
# s=1
# for k in cards:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[0])
#     s+=1
# print(s+1)